# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## 1. Изучим общую информацию о данных. 

### Импорт библиотек

In [1]:
import pandas as pd
import sidetable
from io import BytesIO
import requests

import warnings
warnings.simplefilter('ignore')

from pymystem3 import Mystem
from collections import Counter
from IPython.display import display
from tqdm import tqdm
tqdm.pandas()

### Обзор данных

In [2]:
data = ['data.csv']
path = open('link.txt', 'r').readline()

for file in data:
    try:
        response = requests.get(path + file)
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        path = '/datasets/'

data = pd.read_csv(path + data[0])

In [3]:
def sep_text(text):
    """Функция для разделения текста hr"""
    return print(f'''
===============================================================
{text: ^64}
===============================================================
    ''')

In [4]:
def brows_data(data, name):
    """Отображение основных параметров - df, info, describe, dtypes"""
    data.name = name
    sep_text('DataFrame' + ': ' + name)
    display(data)
    sep_text('Info' + ': ' + name)
    display(data.info())
    sep_text('Describe' + ': ' + name)
    display(data.describe().T)
    sep_text('Dtypes' + ': ' + name)
    display(data.dtypes)

In [5]:
data_dict = {'data':data}

for name, df in tqdm(data_dict.items()):
    brows_data(df, name)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


                        DataFrame: data                         
    


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля



                           Info: data                           
    
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None


                         Describe: data                         
    


,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06



                          Dtypes: data                          
    


children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.47it/s]


### Вывод

Таблица содержит в себе **21525** записей, где в `days_employed`, `total_income` имеются пропуски данных.

Сразу заметим, что в `days_employed` хранится еще и отрицательное значение рабочих дней и тип данных float, в `education` уровень образования записан в различном регистре и в `total_income` - тип float.

### Изучим по отдельности содержание столбцов на выявление аномалий

### Столбец `children`

In [6]:
data['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

### Вывод

Столбец `children` содержит в себе 47 записей с отрицательным значением количества детей и выброс 76 записей с значением 20, что маловероятно, учитывая, что предыдущее ближайшее значение - 5 детей у 9 записей.  

### Столбец `days_employed`

Отсортируем `days_employed`, чтобы отметить, что таблица заполнена отрицательными числами и есть пропуски данных:

In [7]:
data['days_employed'].sort_values().head()

16335   -18388.949901
4299    -17615.563266
7329    -16593.472817
17838   -16264.699501
16825   -16119.687737
Name: days_employed, dtype: float64

In [8]:
data['days_employed'].sort_values(ascending=False).tail()

21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: days_employed, dtype: float64

### Вывод

Столбец `days_employed` содержит в себе:
* отрицательные значения дней стажа 
* пропущенные значения NaN
* данные в типе данных float

### Столбец `dob_years`

Посмотрим, какие значения хранятся в `dob_years`:

In [9]:
data['dob_years'].value_counts().sort_index().head()

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

In [10]:
data['dob_years'].value_counts().sort_index().tail()

71    58
72    33
73     8
74     6
75     1
Name: dob_years, dtype: int64

### Вывод

Получается, что 101 человек с возрастом 0 хочет получить кредит! Не пойдет.

### Столбцы `education` и `education_id`

In [11]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [12]:
data['education_id'].value_counts().sort_index()

0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64

### Вывод

Столбец `education` содержит в себе дублирующие значения из-за разных регистров. В остальном все в порядке.

### Столбцы `family_status` и `family_status_id`

In [13]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [14]:
data['family_status_id'].value_counts().sort_index()

0    12380
1     4177
2      960
3     1195
4     2813
Name: family_status_id, dtype: int64

### Вывод

В `family_status` почти все в порядке, портит общий вид только **Не женат** с заглавной буквы.

### Столбец `gender `

In [15]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Посмотрим, что это за строка с значением XNA:

In [16]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


### Вывод

Отметим, что в данные попала ошибочное значение **XNA**.

### Столбец `income_type `

In [17]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

### Вывод

Данным не требуется обработка.

### Столбец `debt `

In [18]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

### Вывод

С данными все в порядке, никаких ошибок или выбросов.

### Столбец `total_income `

Посмотрим на минимальный и максимальный ежемесячный доход:

In [19]:
data['total_income'].sort_values().head()

14585    20667.263793
13006    21205.280566
16174    21367.648356
1598     21695.101789
14276    21895.614355
Name: total_income, dtype: float64

In [20]:
data['total_income'].sort_values(ascending = False).head()

12412    2.265604e+06
19606    2.200852e+06
9169     1.726276e+06
20809    1.715018e+06
17178    1.711309e+06
Name: total_income, dtype: float64

### Вывод

Значения записаны в типе float и часть значений записаны в экспоненциальном виде.

### Столбец `purpose `

In [21]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Столбец содержит в себе много дубликатов, которые нужно привести к универсальному виду.

### 2. Предобработка данных

### Обработка аномалий в данных

Как уже говорилось выше в столбцах `children, dob_years и gender` встречаются аномалии, напомним их:

1. `children` --- 47 клиентов с количество детей -1, 76 клиентов с 20 детьми (при ближайшем наименьшем значении 5)

2. `dob_years` --- 0 возраст 101 клиента банка

3. `gender` --- 1 неопределенное значение XNA

Исправим данные аномалии и ошибки.

Сделаем фильтр для записей, где значение детей отрицательное и возьмем это значение по модулю:

In [22]:
data.loc[data['children'] < 0, 'children'] = data.loc[data['children'] < 0, 'children'].abs()

Отберем также записи, где значение в столбце `children` равно 20 и посчитаем среднее значении количества детей (без учета записей, где детей 20):

In [23]:
mean_children = data.loc[data['children'] != 20, 'children'].mean()
data.loc[data['children'] == 20, 'children'] = mean_children

Проверим, что получилось после фильтрации данных:

In [24]:
data['children'].value_counts()

0.000000    14149
1.000000     4865
2.000000     2055
3.000000      330
0.474334       76
4.000000       41
5.000000        9
Name: children, dtype: int64

Отфильтруем таблицу по значению 0 столбца возраста и заполним ошибочные данные средним по столбцу (без учета 0 значений):

In [25]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data.loc[data['dob_years'] != 0, 'dob_years'].mean()

In [26]:
data['dob_years'].value_counts().sort_index().head()

19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
Name: dob_years, dtype: int64

Осталось разобраться с неверной записью пола, в связи с тем, что не представляется возможным определить ее примерное значение, удалим данную запись из данных:

In [27]:
data.loc[data['gender'] == 'XNA', 'gender'] = None

In [28]:
data = data.dropna(subset = ['gender'])

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  float64
 1   days_employed     19350 non-null  float64
 2   dob_years         21524 non-null  float64
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.1+ MB


### Вывод

Аномалию с количеством детей исправили при помощи модуля от отрицательного значения.

Неправильный возраст клиента был заполнен среднем арифметическим значением.

Ошибка в данных с полом, к сожалению, восстановлению не подлежит, т.к. невозможно идентифицировать  пол клиента по данным, в связи с чем ошибочная запись была удалена.

### Обработка пропусков

Посчитаем, сколько пропусков в столбцах `days_employed` и  `total_income`:

In [30]:
for column in ['days_employed', 'total_income']:
    print(f'Пропусков в столбце {column}:', data[column].isna().sum())

Пропусков в столбце days_employed: 2174
Пропусков в столбце total_income: 2174


В каждом из столбцов пропущенно одинаковое количество данных --- 2174.

Посчитаем, сколько записей с одновременно пропущенными значениями:

In [31]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Можно сделать вывод, что в **2174** записях пропущено и значение `days_employed` и значение `total_income`.

Значение трудового стажа не требуется для нахождения зависимости, но возьмем модуль от отрицательных значений и заполним пропущенные данные при помощи медианы, чтобы выбросы в данных не сильно влияли:

In [32]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed'].abs()

In [33]:
median_days_emploed = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(median_days_emploed)

Значение ежемесячного дохода требуется для исследования зависимости между доходом и сроком возврата кредита, поэтому найдем среднее значение по роду дейтельности и заполним им пропуски:

In [34]:
mean_total_income_type = data.groupby('income_type')['total_income'].mean()
mean_total_income_type

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.136353
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [35]:
for inc_type in mean_total_income_type.index:
    data.loc[data['total_income'].isna() == True, 'total_income'] = data.loc[data['income_type'] == inc_type,'total_income'].fillna(mean_total_income_type[inc_type])

Проверяем, что пропусков больше не осталось:

In [36]:
data.stb.missing()

,missing,total,percent
children,0,21524,0.0
days_employed,0,21524,0.0
dob_years,0,21524,0.0
education,0,21524,0.0
education_id,0,21524,0.0
family_status,0,21524,0.0
family_status_id,0,21524,0.0
gender,0,21524,0.0
income_type,0,21524,0.0
debt,0,21524,0.0


### Вывод

Пропущенные значения, обнаруженные в столбцах `days_employed` и `total_income` заполнены.

Такие пропуски могут быть связаны с отсутствующими документами о стаже и справке о ежемесячном заработке в банке, которые не предоставил клиент.

Отрицательные значения в `days_employed` заменены на положительные, а пропущенные - на медиану. Значения в `total_income`, напротив, влияют на исследование, поэтому соответствующие отсутствующие значение заменено на среднее арифметическое по данному типу занятости.

### Замена типа данных

Столбцы `days_employed`, `total_income` и `dob_years` имеют тип данных float, изменим это, переведя их в тип данных **int()**:

In [37]:
data['days_employed'] = data['days_employed'].astype(int)

In [38]:
data['total_income'] = data['total_income'].astype(int)

In [39]:
data['dob_years'] = data['dob_years'].astype(int)

In [40]:
data['children'] = data['children'].astype(int)

Проверим, что операция была успешно выполнена:

In [41]:
data.dtypes

children             int32
days_employed        int32
dob_years            int32
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int32
purpose             object
dtype: object

### Вывод

В данном случае тип данных float не требуется, данные хранящиеся в `days_employed`, `total_income` и `dob_years` по смыслу должны быть целыми числами.

### Обработка дубликатов

Перед тем как приступить к поиску дубликатов, приведем к нижнему регистру значения в столбцах, где используется текст c различным написанием регистра:

In [42]:
columns = ['education', 'family_status']
for column in columns:
    data[column] = data[column].str.lower()

Посчитаем количество дубликатов в data:

In [43]:
data.duplicated().sum()

71

Посмотрим сводную информацию по уровню образования:

In [44]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Удалим лишние дубликаты из data и выведем общую информацию о данных:

In [45]:
data = data.drop_duplicates().reset_index(drop = True)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int32 
 1   days_employed     21453 non-null  int32 
 2   dob_years         21453 non-null  int32 
 3   education         21453 non-null  object
 4   education_id      21453 non-null  int64 
 5   family_status     21453 non-null  object
 6   family_status_id  21453 non-null  int64 
 7   gender            21453 non-null  object
 8   income_type       21453 non-null  object
 9   debt              21453 non-null  int64 
 10  total_income      21453 non-null  int32 
 11  purpose           21453 non-null  object
dtypes: int32(4), int64(3), object(5)
memory usage: 1.6+ MB


### Вывод

После приведения текстовымх значений к строчной форме записи, удалось убрать 71 дубликат из data.

### Лемматизация

Выполним лемматизацию целей получения кредита для того, чтобы выделить самые популярные категории.

Запишем уникальные значения столбца `purpose` в отдельный dataframe, чтобы работать не со всеми записями `data` (включая повторы), а лишь с уникальными значениями, тем самым сократив время лемматизации и обработки данных.

In [47]:
unique_purpose_data = pd.DataFrame(data = data['purpose'].unique(), columns = ['purpose'])

Посмотрим информацию о unique_purpose_data:

In [48]:
unique_purpose_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   purpose  38 non-null     object
dtypes: object(1)
memory usage: 432.0+ bytes


Как видим из вывода, у нас есть 38 уникальных значений. Выведем первые 10 строк таблицы:

In [49]:
unique_purpose_data.head(10)

,purpose
0,покупка жилья
1,приобретение автомобиля
2,дополнительное образование
3,сыграть свадьбу
4,операции с жильем
5,образование
6,на проведение свадьбы
7,покупка жилья для семьи
8,покупка недвижимости
9,покупка коммерческой недвижимости


Сделаем лемматизацию "на лету", чтобы найти самые распространенные категории для получения кредита:

In [50]:
unique_purpose_data['lemmas'] = unique_purpose_data['purpose'].progress_apply(Mystem().lemmatize)

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [01:08<00:00,  1.81s/it]


Найдем самые часто встречаемые категории при помощи `Counter`:

In [51]:
Counter(unique_purpose_data['lemmas'].sum()).most_common()

[(' ', 59),
 ('\n', 38),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('подержать', 1),
 ('со', 1),
 ('подержанный', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1)]

Как видно из вывода, можно выделить 5 популярных категорий:
* жилье
* недвижимость
* свадьба
* автомобиль
* образование

Причем, отметим, что жилье и недвижимость можно объединить в одну категорию.

Напишем фукнцию, которая будет принимать текст, делить на леммы и возвращать соответствующую категорию:

In [52]:
def give_purpose_category(text):
    m = Mystem()
    lemma = m.lemmatize(text)
    
    if ('жилье' in lemma) or ('недвижимость' in lemma):
        return 'недвижимость'
    if ('свадьба' in lemma):
        return 'свадьба'
    if ('автомобиль' in lemma):
        return 'автомобиль'
    if ('образование' in lemma):
        return 'образование'

In [53]:
unique_purpose_data['purpose_category'] = unique_purpose_data['purpose'].progress_apply(give_purpose_category)

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:59<00:00,  1.57s/it]


In [54]:
unique_purpose_data.head(10)

,purpose,lemmas,purpose_category
0,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,дополнительное образование,"[дополнительный, , образование, \n]",образование
3,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
4,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
5,образование,"[образование, \n]",образование
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость
8,покупка недвижимости,"[покупка, , недвижимость, \n]",недвижимость
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]",недвижимость


Выделив категории, добавим к data соответствующий новый столбец `purpose_category`:

In [55]:
data = data.merge(unique_purpose_data, on='purpose', how = 'left')

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21452
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int32 
 1   days_employed     21453 non-null  int32 
 2   dob_years         21453 non-null  int32 
 3   education         21453 non-null  object
 4   education_id      21453 non-null  int64 
 5   family_status     21453 non-null  object
 6   family_status_id  21453 non-null  int64 
 7   gender            21453 non-null  object
 8   income_type       21453 non-null  object
 9   debt              21453 non-null  int64 
 10  total_income      21453 non-null  int32 
 11  purpose           21453 non-null  object
 12  lemmas            21453 non-null  object
 13  purpose_category  21453 non-null  object
dtypes: int32(4), int64(3), object(7)
memory usage: 2.1+ MB


Подсчитаем какое распределение целей кредита по категориям:

In [57]:
data['purpose_category'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

### Вывод

Используя лемматизацию удалось разделить цели получения кредита из формата `На покупкой новой недвижимости` в `Недвижимость`, выделив тем самым категории с часто встречаемыми целями кредита.

### Категоризация данных

#### Категории для возрастов

Приведем для удобства наши данные к категориям, например, для деления по возрасту воспользуемся классификацией АПН СССР 1965г. с небольшими поправками (кредит не выдают до 18 лет) и выделим такие группы возрастов как:

* Юные 18-24 лет
* Молодые 25-35 лет
* Взрослые 36-60 лет
* Пожилые 61-75 лет

Реализуем функцию, согласно выше описанной градации, которая будет возвращать соответствующую категорию согласно возрасту:

In [58]:
def age_category(age):
    if 18 <= age <= 24:
        return 'юные'
    elif 25 <= age <= 35:
        return 'молодые'
    elif 36 <= age <= 60:
        return 'взрослые'
    elif 61 <= age <= 75:
        return 'пожилые'
    else:
        return 'другое'

In [59]:
data['years_category'] = data['dob_years'].progress_apply(age_category)

100%|████████████████████████████████████████████████████████████████████████| 21453/21453 [00:00<00:00, 306666.00it/s]


In [60]:
data['years_category'].value_counts()

взрослые    12745
молодые      5708
пожилые      2126
юные          874
Name: years_category, dtype: int64

#### Категории для количества детей в семье

Разделим семьи на количество детей по категориям:
* 0 детей - нет детей
* 1-2 ребенка - обычная семья
* от 3 - многодетная семья

In [61]:
def count_children_and_category(children):
    if children == 0:
        return 'нет детей'
    if 1 <= children <= 2:
        return 'обычная семья'
    if children >= 3:
        return 'многодетная семья'

In [62]:
data['children_category'] = data['children'].progress_apply(count_children_and_category)

100%|████████████████████████████████████████████████████████████████████████| 21453/21453 [00:00<00:00, 282457.17it/s]


In [63]:
data['children_category'].value_counts()

нет детей            14166
обычная семья         6907
многодетная семья      380
Name: children_category, dtype: int64

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21452
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   children           21453 non-null  int32 
 1   days_employed      21453 non-null  int32 
 2   dob_years          21453 non-null  int32 
 3   education          21453 non-null  object
 4   education_id       21453 non-null  int64 
 5   family_status      21453 non-null  object
 6   family_status_id   21453 non-null  int64 
 7   gender             21453 non-null  object
 8   income_type        21453 non-null  object
 9   debt               21453 non-null  int64 
 10  total_income       21453 non-null  int32 
 11  purpose            21453 non-null  object
 12  lemmas             21453 non-null  object
 13  purpose_category   21453 non-null  object
 14  years_category     21453 non-null  object
 15  children_category  21453 non-null  object
dtypes: int32(4), int64(3), object(9)
memory 

#### Категории для ежемесячного заработка

Введем категории для ежемесечного заработка, для этого подсчитаем значения 0%, 25%, 50%, 75%, 100%, где 0% - это минимальный заработок, а 100% - максимальный представленные в данных. 

Сохраним данные в словарь, после чего воспользуемся функцией и разделим столбец `total_income` на категории: 
* бедные (0%-25%),
* ниже среднего (25%-50%), 
* выше среднего (50%-75%),
* богатые (свыше 75%).

In [65]:
money_range = dict(data['total_income'].quantile([0, 0.25, 0.50, 0.75, 1]).astype(int))
money_range

{0.0: 20667, 0.25: 107620, 0.5: 151876, 0.75: 202417, 1.0: 2265604}

In [66]:
def income_category(money):
    if money_range[0.0] <= money <= money_range[0.25]:
        return 'бедные'
    elif money_range[0.25] < money <= money_range[0.5]:
        return 'ниже среднего'
    elif money_range[0.5] < money <= money_range[0.75]:
        return 'выше среднего'
    elif money_range[0.75] < money:
        return 'богатые'
    else:
        return 'другое'

In [67]:
data['income_category'] = data['total_income'].progress_apply(income_category)

100%|████████████████████████████████████████████████████████████████████████| 21453/21453 [00:00<00:00, 193392.70it/s]


Подсчитаем количество значений для каждой категории в новом столбце `income_category`:

In [68]:
data['income_category'].value_counts()

выше среднего    5818
бедные           5364
ниже среднего    5363
богатые          4908
Name: income_category, dtype: int64

### Вывод

Выделив основные категории по столбцам, работать с данными стало намного удобнее.

### 3. Ответим на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Создадим сводную таблицу для того, чтобы ответить на данные вопрос.

Возьмем данные из столбца `children_category, debt, dob_years` и используем функцию `count` на значениях возраста (он заполнен у всех клиентов), чтобы подсчитать количество клиентов, которые имели задолжность / без задолжностей:

In [69]:
children_debt = data.pivot_table(index = 'children_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
children_debt.columns = ['Нет долга', 'Есть долг']
children_debt['Доля должников'] = (children_debt['Есть долг'] / 
                                (children_debt['Есть долг'] + 
                                 children_debt['Нет долга'])).map('{:.1%}'.format)

In [70]:
children_debt.sort_values(by = 'Доля должников', ascending = False)

,Нет долга,Есть долг,Доля должников
children_category,,,
обычная семья,6268,639,9.3%
многодетная семья,349,31,8.2%
нет детей,13095,1071,7.6%


Добавим так же сводную таблицу для категорий по целям кредита у разных семей для составления общей картины:

In [71]:
data.pivot_table(index = 'children_category', columns = 'purpose_category', values = 'dob_years', aggfunc = 'count')

purpose_category,автомобиль,недвижимость,образование,свадьба
children_category,,,,
многодетная семья,72,195,77,36
нет детей,2861,7109,2657,1539
обычная семья,1373,3506,1279,749


### Вывод

Как видно из сводной таблицы, самая большая доля должников у обычных семей (1-2 ребенка в семье), самая наименьшая у семей без детей. 

Такие результаты связаны с дополнительными затратами на ребенка и малым количеством свободных денег, в тоже время доля должников у многодетных семей ниже, чем у обычных, это связано с поддержкой многодетных семей государством, где большую часть материнского капитала можно использовать для покупки недвижимости или получения платного образования детьми, а также меньшем количеством таких многодетных семей.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Выполним шаги, аналогичные раннее, для этого создадим сводную таблицу по семейному положению и возвратом кредита:

In [72]:
family_debt = data.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')
family_debt.columns = ['Нет долга', 'Есть долг']
family_debt['Доля должников'] = (family_debt['Есть долг'] / 
                                (family_debt['Есть долг'] + 
                                 family_debt['Нет долга'])).map('{:.1%}'.format)

Полученные результаты выведем по-убыванию доли должников:

In [73]:
family_debt.sort_values(by = 'Доля должников', ascending = False)

,Нет долга,Есть долг,Доля должников
family_status,,,
не женат / не замужем,2536,274,9.8%
гражданский брак,3762,388,9.3%
женат / замужем,11408,931,7.5%
в разводе,1110,85,7.1%
вдовец / вдова,896,63,6.6%


### Вывод

Зависимость семейного положения и возратом кредита в срок говорит о том, что среди не женатых / не замужних, а также тех, кто в гражданском браке намного выше, чем у тех, кто женат / замужем, в разводе или вдовец / вдова. 

Интерпретировать такой результат можно как умение принять на себя ответственность и навык ведения уже не только личного, но и семейного бюджета. :)

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Рассмотрим сводную таблицу по уровню дохода и возвратом кредита в срок:

In [74]:
income_debt = data.pivot_table(index = 'income_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
income_debt.columns = ['Нет долга', 'Есть долг']
income_debt['Доля должников'] = (income_debt['Есть долг'] / 
                                (income_debt['Есть долг'] + 
                                 income_debt['Нет долга'])).map('{:.1%}'.format)

In [75]:
income_debt.sort_values(by = 'Доля должников', ascending = False)

,Нет долга,Есть долг,Доля должников
income_category,,,
ниже среднего,4888,475,8.9%
выше среднего,5324,494,8.5%
бедные,4937,427,8.0%
богатые,4563,345,7.0%


Расширим нашу таблицу, добавив столбец с целями кредита, чтобы получить более полную картину:

In [76]:
income_purpose_debt = data.pivot_table(index = ['income_category', 'purpose_category'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
income_purpose_debt.columns = ['Нет долга', 'Есть долг']
income_purpose_debt['Доля должников'] = (income_purpose_debt['Есть долг'] / 
                                (income_purpose_debt['Есть долг'] + 
                                 income_purpose_debt['Нет долга'])).map('{:.1%}'.format)

In [77]:
income_purpose_debt

Нет долга  Есть долг Доля должников
income_category purpose_category                                     
бедные          автомобиль              943        101           9.7%
                недвижимость           2495        181           6.8%
                образование             951         91           8.7%
                свадьба                 548         54           9.0%
богатые         автомобиль              927         83           8.2%
                недвижимость           2334        166           6.6%
                образование             802         70           8.0%
                свадьба                 500         26           4.9%
выше среднего   автомобиль             1009        108           9.7%
                недвижимость           2759        225           7.5%
                образование             989        108           9.8%
                свадьба                 567         53           8.5%
ниже среднего   автомобиль             1024        111           9.8%
                недвижимость           2440        210           7.9%
                образование             901        101          10.1%
                свадьба                 523         53           9.2%

### Вывод

Как видим из данных наибольший процент должников среди тех, у кого заработок ниже и выше среднего. В основном, такой процент должников из-за кредитов на образование и автомобили, когда как у бедных лидируют кредиты на автомобили и свадьбу, богатых - автомобили и образование. Сейчас без авто - никуда :)

Самое интересное, что на недвижимость в большинстве категорий самый низкий процент должников - это связано с низком процентом, большим сроком ипотеки и низкими ежемесячными платежами, также психологической составлящей - остаться без жилья страшнее, чем без автомобиля или платного образования.

Свадьбы в кредит - это зло, богатые люди это понимают.

- **Как разные цели кредита влияют на его возврат в срок?**

In [78]:
purpose_debt = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
purpose_debt.columns = ['Нет долга', 'Есть долг']
purpose_debt['Доля должников'] = (purpose_debt['Есть долг'] / 
                                (purpose_debt['Есть долг'] + 
                                 purpose_debt['Нет долга'])).map('{:.1%}'.format)

In [79]:
purpose_debt.sort_values(by = 'Доля должников', ascending = False)

,Нет долга,Есть долг,Доля должников
purpose_category,,,
автомобиль,3903,403,9.4%
образование,3643,370,9.2%
свадьба,2138,186,8.0%
недвижимость,10028,782,7.2%


### Вывод

Автомобиль требует нетолько выплаты платежей по кредиту, но также несет в себе дополнительные расходы: ремонт, страховка, обслуживание, топливо - неудивительно, что взяв кредит, клиенты не оценивают косвенные затраты на автомобиль.

Недвижимость же находится в самом конце списка, что говорит о более доступных платежах по ипотеке и не таких регулярных затратах на жилье.

### 4. Общий вывод

Подводя итоги, можно сказать, что у не замужних / не женатых, а также состоящих в гражданском браке доля должников выше, чем у остальных:

In [80]:
family_debt.sort_values(by = 'Доля должников', ascending = False)

,Нет долга,Есть долг,Доля должников
family_status,,,
не женат / не замужем,2536,274,9.8%
гражданский брак,3762,388,9.3%
женат / замужем,11408,931,7.5%
в разводе,1110,85,7.1%
вдовец / вдова,896,63,6.6%


Количество детей влияет на доля должников, семьи без детей - вероятнее вернут кредит, чем семьи с 1-2 детьми, многодетные семьи - пользуются льготами государства, поэтому доля должников среди них ниже, чем у обычных семей.

In [81]:
children_debt.sort_values(by = 'Доля должников')

,Нет долга,Есть долг,Доля должников
children_category,,,
нет детей,13095,1071,7.6%
многодетная семья,349,31,8.2%
обычная семья,6268,639,9.3%
